In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib as ta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
X_train = pd.read_csv('Train.csv')
y_train = X_train['up_down']
X_train = X_train.drop(columns = ['up_down','Gmt time'])

X_test = pd.read_csv('Test.csv')
y_test = X_test['up_down']
X_test = X_test.drop(columns = ['up_down','Gmt time'])

In [5]:
X_train.head()

,Open,High,Low,Close,Volume,body,upper_tail,lower_tail,SMA_50,SMA_20,...,lag_return_92,return_93,lag_return_93,return_94,lag_return_94,return_95,lag_return_95,return_96,lag_return_96,up_down
0,1.35492,1.35506,1.35397,1.35408,156.70,-8.4,1.4,1.1,1.356870,1.356554,...,-0.000051,-0.005150,0.000125,-0.005026,0.000927,-0.004104,-0.000581,-0.004682,0.000338,0
1,1.29505,1.29505,1.29505,1.29505,0.00,0.0,0.0,0.0,1.295050,1.295050,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,1.27300,1.27300,1.27300,1.27300,0.00,0.0,0.0,0.0,1.273000,1.273000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,1.11490,1.11534,1.11489,1.11511,878.11,2.1,2.3,0.1,1.115236,1.114623,...,0.000798,0.000081,-0.000636,-0.000556,-0.000609,-0.001164,0.000421,-0.000744,-0.000994,1
4,1.39988,1.39991,1.39956,1.39988,525.35,0.0,0.3,3.2,1.397725,1.398727,...,0.000512,0.008792,-0.000375,0.008414,0.000209,0.008624,-0.000267,0.008356,-0.000144,0


In [ ]:
# scaling Volume
import sklearn.preprocessing as preprocessing
scaler = preprocessing.StandardScaler()
vol_scale_param = scaler.fit(X_train['Volume'].values.reshape(-1,1))

X_train['Volume_scaled'] = scaler.fit_transform(X_train['Volume'].values.reshape(-1,1), vol_scale_param)
X_train = X_train.drop(columns = ['Volume'])

X_test['Volume_scaled'] = scaler.fit_transform(X_test['Volume'].values.reshape(-1,1), vol_scale_param)
X_test = X_test.drop(columns = ['Volume'])


In [ ]:
# feature selection
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

select = RFE(RandomForestClassifier(n_estimators=100, random_state=42), n_features_to_select=40)
#select = RFE(LogisticRegression(penalty="l1"), n_features_to_select=40)

select.fit(X_train, y_train)
# visualize the selected features:
mask = select.get_support()
plt.matshow(mask.reshape(1, -1), cmap='gray_r')



In [ ]:
X_train_rfe = select.transform(X_train)
X_test_rfe = select.transform(X_test)

LogisticRegression().fit(X_train_rfe, y_train).score(X_test_rfe, y_test)


In [ ]:
select.score(X_test, y_test)